# Pseudorandom generators:

L'idea è quella di ottenere uno schema di encryption <b>EAV-sicuro</b>, basato su OTP, ma con chiavi più corte del plaintext.

Per avere una chiave corta, è necessario espanderla:\
$k \leftarrow \{0,1\}^m \quad \ m<<n \quad \ G(k) = r \in \{0, 1\}^n$

$E_k(x) = r ⊕ k = G(K) ⊕ x$

$G$ è una funzione <b>deterministica</b> che espande $k$: $\quad k$ è il <b>seed</b> di un generatore pseudorandom

Vorrei che $r$ fosse indistinguibile da una sequenza di $n$ bit random (per un attaccante polinomiale)

### Complessità di Kolmogorov:

Una sequenza di bit è <b>random</b> se non esiste un programma che la genera che sia sostanzialmente più corto della sequenza stessa:\
in altre parole, se ho una sequenza e un programma che al genera, se il programam è più corto della sequenza, la sequenza non è random.

Kolmagorov considera anche i programmi <b>inefficienti</b>, noi restringiamo il campo soltanto a quelli <b>efficienti</b>.

### Approccio crittografico:

Non consideriamo sequenza di bit random, ma una <b>distribuzione</b> random di sequenze.\
Non consideriamo, inoltre, i programmi che generano le sequenze: definiamo come random una distribuzione quando supera tutti i possibili test statistici.

## Distinguishers:

Un <b>distinguisher</b> è un <b>algoritmo $PPTIME$</b> che simula i test statistici: prende in input una sequenza di bit e restituisce in output 0 se la sequenza <b>non</b> sembra random oppure 1 se la sequenza in input sembra random.

Un generatore pseudorandom $G$ è buono se il miglior distinguisher può soltanto <b>tirare a indovinare</b>.

## Generatore pseudorandom:

$G$ algoritmo <b>deterministico</b> $PTIME$ da $\{0,1\}^n$ a $\{0,1\}^{l(n)}$

$G$ è un <b> generatore pseudorandom</b> (PRG) se: 

1. $\forall n \ \ l(n) > n \quad$ <i>espansività</i>
2. $Pr(PRG_{M,G}(n) = 1) \leq 1/2 + negl(n) \quad \forall M \in PPTIME$

## Esperimento $PRG_{M,G}(n)$

$1. \quad A: \ \ b \leftarrow \{0,1\}$
- $if \ \ b==0: \quad r \leftarrow \{0,1\}^{l(n)} \quad$ <i>completamente random</i> 
- $if \ \ b==1: \quad s \leftarrow \{0,1\}^{n} \quad r:=G(s) \quad$ <i>soltanto il <b>seed</b> è random, $r$ è <b>deterministico</b></i>

$2. \quad A \rightarrow M: \ \ r$\
$3. \quad M: \ \ b_m \quad$ <i>$M$ deve indovinare $b_m$

$PRG_{M,G}(n) = 1 \ \ se \ \ b_m=b$ 

$M$ deve distinguere <b>randomness</b> ($b=0$) da <b>pseudorandomness</b> ($b=1$)

<b>Attenzione</b>: bisogna controllare che M (ovvero l'attaccante o, in questo caso il <b>distinguisher</b>, sia <b>polinomiale</b> (efficiente) e anche la condizione di <b>espansività</b>

#### Esercizio #1:

$G(s)=sb' \quad b' = ⊕_{i=1...n}^{n} s[i]$

$M: \quad 0 \ \ if \ ⊕_{i=1...n}^{n} r[i] \neq r[n+1] \ \ else: \ \ 1 $

$G$ <b>non</b> è pseudorandom

Praticamente, $M$ sta facendo gli stessi passi di $G$ visto che può vedere il <b>seed</b> (restituito in chiaro) con lo XOR di tutti i bit del seed messo come ultimo.

#### Esercizio #2:

$G(s)=ss$

$l(n)>n \quad$ <i>soddisfa <b>espansività</b></i> 

strategia del distinguisher: $\quad 1 \ \ if \ w[1..n] == w[n+1..2n] \ \ 0 \ \ otherwise$

$Pr(PRG_{D,G}(n)=1) = \frac{1}{2} (Pr(b_D=0 | b=0)Pr(b_D=1 | b=1))$

$Pr(b_D=0|b=0) = 1 - \frac{1}{2^n-2} \quad$ <i>i casi sfortunati dove $r=0^n$ o $r=1^n$</i> \
$Pr(b_D=1|b=1) = 1$ 

$Pr(PRG_{D,G}(n)=1) = \frac{1}{2}(1 - \frac{1}{2^n-2} + 1) > \frac{1}{2} + negl(n)$

In [25]:
import random

def g(n): 
    b = random.randint(0,1)
    r = ''
    s = ''
    
    if(b==0):
        for i in range(2*n):
            r += str(random.randint(0,1))
    else:
        for i in range(n):
            s += str(random.randint(0,1))
        r = s + s

    print("b is equal to: ", b)
    print("R is equal to: ", r)
    print("s is equal to: ", s)

    return r

def distinguisher(r):
    return r[:len(r)//2] != r[-len(r)//2:] # if the two halves are equal, say 0 else 1: this is why != instead of ==

n=10
distinguisher(g(n))

# note: if b==0 and output is True: M wins (because it means G is true that it is a PRG)
# note: if b==1 and output is False: M wins (because it means G is false that it is a PRG)

### Esercizio #3:

$G(s) = s || f(s)$

$|f(s)| > 0 \quad f \in PTIME$

$D$ (o $M$) conosce la lunghezza di $s$: $n$ e conosce anche la funzione $f$ (principio di Kerckhoffs): ovviamente non può però conoscere l'esito delle generazioni random private di Alice.

$1. \quad A: \ \ b \leftarrow \{0,1\}$
- $if \ \ b==0: \quad r \leftarrow \{0,1\}^{l(n)} \quad$ 
- $if \ \ b==1: \quad s \leftarrow \{0,1\}^{n} \quad r:=G(s) = s||f(s) \quad$ 

$2. \quad A \rightarrow M: \ \ r=w=w_1..w_nw_{n+1}..w_m \quad dove \ \ |w|=m=l(n)$

$w_1..w_n = s$\
$w_{n+1}..w_m = f(s)$\

$3. \quad M: \ \ b_m = \{(w_{n+1}..w_m \ == \ f(w_1..w_n)) \ \ ? \ \ 1 \ \ : \ \  0 \}\quad$ <i>$M$ calcola $f(s)$ perchè, conoscendo $n=|s|$, è in grado di dividere la stringa come vuole</i>

$Pr(b_m=0 | b=0) = Pr((f(w_1..w_n) \neq w_{n+1}..w_m) | randomness) = 1 - \frac{1}{2^{m-n}} \quad$ <i>indovina quasi sempre, tranne nel caso sfortunato dove, per caso, la seconda parte è il risultato della funzione $f$ applicata alla prima parte</i>\
$Pr(b_m=1 | b=1) = 1 \quad$ <i>indovina sempre</i>

$G$ <b>non</b> è pseudorandom

## Pseudo-OTP ($_pOTP$):

Sia $G \in \{0,1\}^n \rightarrow \{0,1\}^{l(n)}$

$_pOTP = (Gen,E_k,D_k)$

$Gen(1^n) = \{k \leftarrow \{0,1\}^n\}$\
$E_k(x) = G(k) ⊕ x$\
$D_k(y) = G(k) ⊕ y$

$x,y \in \{0,1\}^{l(n)}$

### Teorema:

$G$ è $PRG \rightarrow$ $_pOTP \ $ è $\ EAV$-sicuro